## Importing packages needed

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob
from pathlib import Path

## Importing FitBit Data

In [2]:
filepath = r'C:\Users\63sha\OneDrive\Desktop\Springboard\Capstone_2\Fitabase Data 4.12.16-5.12.16'

files = glob.glob( filepath + "\*.csv")

# creating dictionary to name and store all csv files

df_dict = dict()

for filename in files:
    
    name_df = os.path.basename(filename)
    name_df = name_df.replace('_merged.csv','')
    df_dict[name_df] = pd.read_csv(filename)




In [3]:
# list of our dataframes

print(df_dict.keys())

dict_keys(['dailyActivity', 'dailyCalories', 'dailyIntensities', 'dailySteps', 'heartrate_seconds', 'hourlyCalories', 'hourlyIntensities', 'hourlySteps', 'minuteCaloriesNarrow', 'minuteCaloriesWide', 'minuteIntensitiesNarrow', 'minuteIntensitiesWide', 'minuteMETsNarrow', 'minuteSleep', 'minuteStepsNarrow', 'minuteStepsWide', 'sleepDay', 'weightLogInfo'])


## Our first dataframe

In [4]:
# Take a look at our first dataframe

print(df_dict['dailyActivity'])

             Id ActivityDate  TotalSteps  TotalDistance  TrackerDistance  \
0    1503960366    4/12/2016       13162       8.500000         8.500000   
1    1503960366    4/13/2016       10735       6.970000         6.970000   
2    1503960366    4/14/2016       10460       6.740000         6.740000   
3    1503960366    4/15/2016        9762       6.280000         6.280000   
4    1503960366    4/16/2016       12669       8.160000         8.160000   
..          ...          ...         ...            ...              ...   
935  8877689391     5/8/2016       10686       8.110000         8.110000   
936  8877689391     5/9/2016       20226      18.250000        18.250000   
937  8877689391    5/10/2016       10733       8.150000         8.150000   
938  8877689391    5/11/2016       21420      19.559999        19.559999   
939  8877689391    5/12/2016        8064       6.120000         6.120000   

     LoggedActivitiesDistance  VeryActiveDistance  ModeratelyActiveDistance  \
0       

In [5]:
# naming and separating our first dataframe from our dictionary

dirtyDA = df_dict['dailyActivity']

In [6]:
# Checking for any missing values

counts = dirtyDA.isna().sum()
print(counts.sort_values())

Id                          0
ActivityDate                0
TotalSteps                  0
TotalDistance               0
TrackerDistance             0
LoggedActivitiesDistance    0
VeryActiveDistance          0
ModeratelyActiveDistance    0
LightActiveDistance         0
SedentaryActiveDistance     0
VeryActiveMinutes           0
FairlyActiveMinutes         0
LightlyActiveMinutes        0
SedentaryMinutes            0
Calories                    0
dtype: int64


In [7]:
# Since several of our columns are intergers, we want to check and see howe many rows we got where the data shows zero and 
# determine whether this number is accurate for our variable

for column_name in dirtyDA.columns:
    column = dirtyDA[column_name]
    # Get the count of Zeros in column 
    count = (column == 0).sum()
    print('Count of zeros in column ', column_name, ' is : ', count)

Count of zeros in column  Id  is :  0
Count of zeros in column  ActivityDate  is :  0
Count of zeros in column  TotalSteps  is :  77
Count of zeros in column  TotalDistance  is :  78
Count of zeros in column  TrackerDistance  is :  78
Count of zeros in column  LoggedActivitiesDistance  is :  908
Count of zeros in column  VeryActiveDistance  is :  413
Count of zeros in column  ModeratelyActiveDistance  is :  386
Count of zeros in column  LightActiveDistance  is :  85
Count of zeros in column  SedentaryActiveDistance  is :  858
Count of zeros in column  VeryActiveMinutes  is :  409
Count of zeros in column  FairlyActiveMinutes  is :  384
Count of zeros in column  LightlyActiveMinutes  is :  84
Count of zeros in column  SedentaryMinutes  is :  1
Count of zeros in column  Calories  is :  4


In [8]:
# Several columns show zeros, with a couple having the majority of data as zero. SOme of this may be accurate while other
# columns might not be. We'll take a look at the Logged activity distance and see what these rows can show us

In [9]:
# This particular vcariable tracks when the user logs that they are exercising in some manner and has to be manuallly 
# initiated by user. Looking at our describe method on this column, i dont think this will help our predictive model with so
# little real data.

dirtyDA['LoggedActivitiesDistance'].describe()

count    940.000000
mean       0.108171
std        0.619897
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        4.942142
Name: LoggedActivitiesDistance, dtype: float64

In [10]:
# dropping the column

dirtyDA = dirtyDA.drop('LoggedActivitiesDistance', axis=1)

In [11]:
# looking at the amount of data we are still working with

dirtyDA.shape

(940, 14)

In [12]:
# checking for correct data types

print(dirtyDA.dtypes)

Id                            int64
ActivityDate                 object
TotalSteps                    int64
TotalDistance               float64
TrackerDistance             float64
VeryActiveDistance          float64
ModeratelyActiveDistance    float64
LightActiveDistance         float64
SedentaryActiveDistance     float64
VeryActiveMinutes             int64
FairlyActiveMinutes           int64
LightlyActiveMinutes          int64
SedentaryMinutes              int64
Calories                      int64
dtype: object


In [13]:
# These two columns should be equal unless gps loses our user's location, the system then has to estimate distance based on
# steps. To keep the data as accurate as possible, we are only gonna use the tracker dsistance and only keeping rows where these
# rows are equal.

dirtyDA = dirtyDA[dirtyDA['TotalDistance'] == dirtyDA['TrackerDistance']]

In [14]:
# dropping the column

dirtyDA = dirtyDA.drop('TotalDistance', axis=1)

In [15]:
# checking our data, we removed one column and 15 rows where the tracker distance and total distance did not match

dirtyDA.shape

(925, 13)

In [16]:
# taking a glance at our dataframe

dirtyDA.head()

,Id,ActivityDate,TotalSteps,TrackerDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [17]:
# changing the ID data type to string from interger

dirtyDA['Id'] = dirtyDA.Id.astype(str)

In [18]:
# changing type to a datetime

dirtyDA['ActivityDate'] = pd.to_datetime(dirtyDA['ActivityDate'])

In [19]:
# Checking our changes

dirtyDA.dtypes

Id                                  object
ActivityDate                datetime64[ns]
TotalSteps                           int64
TrackerDistance                    float64
VeryActiveDistance                 float64
ModeratelyActiveDistance           float64
LightActiveDistance                float64
SedentaryActiveDistance            float64
VeryActiveMinutes                    int64
FairlyActiveMinutes                  int64
LightlyActiveMinutes                 int64
SedentaryMinutes                     int64
Calories                             int64
dtype: object

In [20]:
# determining how many users are included in our data. 33 users have uploaded data into our activity file

dirtyDA.Id.nunique()

33

In [21]:
# the users have given us data over 31 days

dirtyDA.ActivityDate.nunique()

31

In [22]:
# taking another look at the zero's in columns.

for column_name in dirtyDA.columns:
    column = dirtyDA[column_name]
    # Get the count of Zeros in column 
    count = (column == 0).sum()
    print('Count of zeros in column ', column_name, ' is : ', count)

Count of zeros in column  Id  is :  0
Count of zeros in column  ActivityDate  is :  0
Count of zeros in column  TotalSteps  is :  77
Count of zeros in column  TrackerDistance  is :  78
Count of zeros in column  VeryActiveDistance  is :  413
Count of zeros in column  ModeratelyActiveDistance  is :  386
Count of zeros in column  LightActiveDistance  is :  85
Count of zeros in column  SedentaryActiveDistance  is :  846
Count of zeros in column  VeryActiveMinutes  is :  409
Count of zeros in column  FairlyActiveMinutes  is :  384
Count of zeros in column  LightlyActiveMinutes  is :  84
Count of zeros in column  SedentaryMinutes  is :  1
Count of zeros in column  Calories  is :  4


In [23]:
# It seems that this data is either incorrect or the user forgot to wear their device for the particular day.
# This data has mostly zeros and wouldnt help us with our model.

print(dirtyDA[dirtyDA['TotalSteps'] == 0].head())

             Id ActivityDate  TotalSteps  TrackerDistance  VeryActiveDistance  \
30   1503960366   2016-05-12           0              0.0                 0.0   
104  1844505072   2016-04-24           0              0.0                 0.0   
105  1844505072   2016-04-25           0              0.0                 0.0   
106  1844505072   2016-04-26           0              0.0                 0.0   
112  1844505072   2016-05-02           0              0.0                 0.0   

     ModeratelyActiveDistance  LightActiveDistance  SedentaryActiveDistance  \
30                        0.0                  0.0                      0.0   
104                       0.0                  0.0                      0.0   
105                       0.0                  0.0                      0.0   
106                       0.0                  0.0                      0.0   
112                       0.0                  0.0                      0.0   

     VeryActiveMinutes  FairlyActiveMi

In [24]:
# dropping the rows with zero steps

dirtyDA = dirtyDA[dirtyDA['TotalSteps'] != 0]

In [25]:
# our 940 rows has now been reduced to 848

dirtyDA.shape

(848, 13)

In [26]:
# checking to make sure the data was dropped

for column_name in dirtyDA.columns:
    column = dirtyDA[column_name]
    # Get the count of Zeros in column 
    count = (column == 0).sum()
    print('Count of zeros in column ', column_name, ' is : ', count)

Count of zeros in column  Id  is :  0
Count of zeros in column  ActivityDate  is :  0
Count of zeros in column  TotalSteps  is :  0
Count of zeros in column  TrackerDistance  is :  1
Count of zeros in column  VeryActiveDistance  is :  336
Count of zeros in column  ModeratelyActiveDistance  is :  309
Count of zeros in column  LightActiveDistance  is :  8
Count of zeros in column  SedentaryActiveDistance  is :  769
Count of zeros in column  VeryActiveMinutes  is :  333
Count of zeros in column  FairlyActiveMinutes  is :  307
Count of zeros in column  LightlyActiveMinutes  is :  7
Count of zeros in column  SedentaryMinutes  is :  1
Count of zeros in column  Calories  is :  0


In [27]:
# checking the lone row that shows zero distance. Seems this user only took 4 step on this day???

print(dirtyDA[dirtyDA['TrackerDistance'] == 0 ])

             Id ActivityDate  TotalSteps  TrackerDistance  VeryActiveDistance  \
107  1844505072   2016-04-27           4              0.0                 0.0   

     ModeratelyActiveDistance  LightActiveDistance  SedentaryActiveDistance  \
107                       0.0                  0.0                      0.0   

     VeryActiveMinutes  FairlyActiveMinutes  LightlyActiveMinutes  \
107                  0                    0                     1   

     SedentaryMinutes  Calories  
107              1439      1348  


In [28]:
# Lets take a look at the days where steps are less than 50, counting how many there are and what the other columns show.
# I decided to keep this data for now until we see what other data weve collected and how much we have. This could also give
# us a base calorie count.

print(dirtyDA[dirtyDA['TotalSteps'] < 50 ])

             Id ActivityDate  TotalSteps  TrackerDistance  VeryActiveDistance  \
100  1844505072   2016-04-20           8             0.01                 0.0   
107  1844505072   2016-04-27           4             0.00                 0.0   
116  1844505072   2016-05-06          44             0.03                 0.0   
264  2347167796   2016-04-29          42             0.03                 0.0   
351  4020332650   2016-04-17          16             0.01                 0.0   
386  4319703577   2016-04-17          29             0.02                 0.0   
411  4319703577   2016-05-12          17             0.01                 0.0   
675  6775888955   2016-05-03           9             0.01                 0.0   
741  7086361926   2016-04-16          31             0.01                 0.0   

     ModeratelyActiveDistance  LightActiveDistance  SedentaryActiveDistance  \
100                       0.0                 0.01                      0.0   
107                       0.0  

In [29]:
# Description of our cleaner data

dirtyDA.describe()

,TotalSteps,TrackerDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
count,848.000000,848.000000,848.000000,848.000000,848.000000,848.000000,848.000000,848.000000,848.000000,848.000000,848.000000
mean,8213.482311,5.899363,1.592241,0.609210,3.610967,0.001509,22.518868,14.633255,208.333726,955.579009,2354.501179
std,4709.291342,3.698350,2.737739,0.906956,1.854686,0.005818,33.716116,20.538355,96.569032,281.919552,705.688206
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.000000
25%,4824.750000,3.327500,0.000000,0.000000,2.327500,0.000000,0.000000,0.000000,145.000000,721.000000,1852.750000
50%,7911.500000,5.540000,0.370000,0.280000,3.550000,0.000000,6.000000,8.000000,206.000000,1020.000000,2205.000000
75%,10821.000000,7.770000,2.142500,0.850000,4.880000,0.000000,33.000000,20.250000,270.000000,1191.000000,2819.750000
max,36019.000000,28.030001,21.920000,6.480000,10.710000,0.070000,210.000000,143.000000,518.000000,1440.000000,4900.000000


In [30]:
# listing our list of dataframes

print(df_dict.keys())

dict_keys(['dailyActivity', 'dailyCalories', 'dailyIntensities', 'dailySteps', 'heartrate_seconds', 'hourlyCalories', 'hourlyIntensities', 'hourlySteps', 'minuteCaloriesNarrow', 'minuteCaloriesWide', 'minuteIntensitiesNarrow', 'minuteIntensitiesWide', 'minuteMETsNarrow', 'minuteSleep', 'minuteStepsNarrow', 'minuteStepsWide', 'sleepDay', 'weightLogInfo'])


## dataframe_2

In [31]:
# the next dataframe

dirtyDC = df_dict['dailyCalories']

In [32]:
dirtyDC.shape

(940, 3)

In [33]:
# This data looks to be a duplicate of the data we already have in our daily activity dataframe

dirtyDC.describe()

,Id,Calories
count,9.400000e+02,940.000000
mean,4.855407e+09,2303.609574
std,2.424805e+09,718.166862
min,1.503960e+09,0.000000
25%,2.320127e+09,1828.500000
50%,4.445115e+09,2134.000000
75%,6.962181e+09,2793.250000
max,8.877689e+09,4900.000000


In [34]:
dirtyDC.Id.nunique()

33

In [35]:
# this data matches columns already in our cleaned dataframe. we can move past it

dirtyDC.head()

,Id,ActivityDay,Calories
0,1503960366,4/12/2016,1985
1,1503960366,4/13/2016,1797
2,1503960366,4/14/2016,1776
3,1503960366,4/15/2016,1745
4,1503960366,4/16/2016,1863


In [36]:
# Checking out the next dataframe. again it seems to be duplicate data, so we will move on

dirtyDI = df_dict['dailyIntensities']

In [37]:
dirtyDI.head()

,Id,ActivityDay,SedentaryMinutes,LightlyActiveMinutes,FairlyActiveMinutes,VeryActiveMinutes,SedentaryActiveDistance,LightActiveDistance,ModeratelyActiveDistance,VeryActiveDistance
0,1503960366,4/12/2016,728,328,13,25,0.0,6.06,0.55,1.88
1,1503960366,4/13/2016,776,217,19,21,0.0,4.71,0.69,1.57
2,1503960366,4/14/2016,1218,181,11,30,0.0,3.91,0.40,2.44
3,1503960366,4/15/2016,726,209,34,29,0.0,2.83,1.26,2.14
4,1503960366,4/16/2016,773,221,10,36,0.0,5.04,0.41,2.71


## Heart rate data

In [38]:
# heartrate dataframe

dirtyHR = df_dict['heartrate_seconds']

In [39]:
dirtyHR.head()

,Id,Time,Value
0,2022484408,4/12/2016 7:21:00 AM,97
1,2022484408,4/12/2016 7:21:05 AM,102
2,2022484408,4/12/2016 7:21:10 AM,105
3,2022484408,4/12/2016 7:21:20 AM,103
4,2022484408,4/12/2016 7:21:25 AM,101


In [40]:
# this dataframe has only 3 columns but is made up of over 2 million rows of heart rate data taken from users every 5 minutes.

dirtyHR.shape

(2483658, 3)

In [41]:
# this data is made up of 14 different users

dirtyHR['Id'].nunique()

14

In [42]:
dirtyHR.dtypes

Id        int64
Time     object
Value     int64
dtype: object

In [43]:
dirtyHR['Id'] = dirtyHR.Id.astype(str)

In [44]:
# we are agian changing data types to the correct ones

dirtyHR['Time'] = pd.to_datetime(dirtyHR['Time'])

In [45]:
dirtyHR.head()

,Id,Time,Value
0,2022484408,2016-04-12 07:21:00,97
1,2022484408,2016-04-12 07:21:05,102
2,2022484408,2016-04-12 07:21:10,105
3,2022484408,2016-04-12 07:21:20,103
4,2022484408,2016-04-12 07:21:25,101


In [46]:
# we are gonna group by the Id and the Day, finding the mean heart rate for that day to simplify our data

dirtyHR = dirtyHR.groupby(['Id', dirtyHR['Time'].dt.date])['Value'].mean()

In [47]:
# after the aggregation

dirtyHR.head()

Id          Time      
2022484408  2016-04-12    75.804177
            2016-04-13    80.337584
            2016-04-14    72.628597
            2016-04-15    80.437382
            2016-04-16    75.960547
Name: Value, dtype: float64

In [48]:
dirtyHR.shape

(334,)

In [49]:
type(dirtyHR)

pandas.core.series.Series

In [50]:
# our dataframe was converted to another type, so we returned it to a dataframe

pd.DataFrame(dirtyHR)

Value
Id         Time                 
2022484408 2016-04-12  75.804177
           2016-04-13  80.337584
           2016-04-14  72.628597
           2016-04-15  80.437382
           2016-04-16  75.960547
...                          ...
8877689391 2016-05-08  72.550523
           2016-05-09  89.615738
           2016-05-10  71.544377
           2016-05-11  89.149122
           2016-05-12  71.087439

[334 rows x 1 columns]

In [51]:
# Resetting index 

dirtyHR = dirtyHR.reset_index()

In [52]:
# our new dataframe with the mean heart rate of each day

dirtyHR.head()

,Id,Time,Value
0,2022484408,2016-04-12,75.804177
1,2022484408,2016-04-13,80.337584
2,2022484408,2016-04-14,72.628597
3,2022484408,2016-04-15,80.437382
4,2022484408,2016-04-16,75.960547


In [53]:
dirtyHR['Time'] = pd.to_datetime(dirtyHR['Time'])

## Merging Data

In [54]:
# merging our two cleaned dataframes of heart rate and activity data

cleanAHR = dirtyDA.merge(dirtyHR, left_on= ['Id', 'ActivityDate'], right_on= ['Id', 'Time'], how= 'left')

In [55]:
# the new dataframe

cleanAHR.head()

,Id,ActivityDate,TotalSteps,TrackerDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,Time,Value
0,1503960366,2016-04-12,13162,8.50,1.88,0.55,6.06,0.0,25,13,328,728,1985,NaT,NaN
1,1503960366,2016-04-13,10735,6.97,1.57,0.69,4.71,0.0,21,19,217,776,1797,NaT,NaN
2,1503960366,2016-04-14,10460,6.74,2.44,0.40,3.91,0.0,30,11,181,1218,1776,NaT,NaN
3,1503960366,2016-04-15,9762,6.28,2.14,1.26,2.83,0.0,29,34,209,726,1745,NaT,NaN
4,1503960366,2016-04-16,12669,8.16,2.71,0.41,5.04,0.0,36,10,221,773,1863,NaT,NaN


In [56]:
# it looks like our heart rate data is missing for over 60% of our data rows. We will keep this data and see if our model can
# improve with it 

for column_name in cleanAHR.columns:
    column = cleanAHR[column_name]
    # Get the count of Zeros in column 
    count = (column).isna().sum()
    print('Count of NAs in column ', column_name, ' is : ', count)

Count of NAs in column  Id  is :  0
Count of NAs in column  ActivityDate  is :  0
Count of NAs in column  TotalSteps  is :  0
Count of NAs in column  TrackerDistance  is :  0
Count of NAs in column  VeryActiveDistance  is :  0
Count of NAs in column  ModeratelyActiveDistance  is :  0
Count of NAs in column  LightActiveDistance  is :  0
Count of NAs in column  SedentaryActiveDistance  is :  0
Count of NAs in column  VeryActiveMinutes  is :  0
Count of NAs in column  FairlyActiveMinutes  is :  0
Count of NAs in column  LightlyActiveMinutes  is :  0
Count of NAs in column  SedentaryMinutes  is :  0
Count of NAs in column  Calories  is :  0
Count of NAs in column  Time  is :  531
Count of NAs in column  Value  is :  531


In [57]:
cleanAHR.shape

(848, 15)

In [58]:
# still have our 33 users

cleanAHR['Id'].nunique()

33

In [59]:
print(df_dict.keys())

dict_keys(['dailyActivity', 'dailyCalories', 'dailyIntensities', 'dailySteps', 'heartrate_seconds', 'hourlyCalories', 'hourlyIntensities', 'hourlySteps', 'minuteCaloriesNarrow', 'minuteCaloriesWide', 'minuteIntensitiesNarrow', 'minuteIntensitiesWide', 'minuteMETsNarrow', 'minuteSleep', 'minuteStepsNarrow', 'minuteStepsWide', 'sleepDay', 'weightLogInfo'])


##  checking out other dataframes

In [60]:
dirtyHC = df_dict['hourlyCalories']

In [61]:
dirtyHC.head()

,Id,ActivityHour,Calories
0,1503960366,4/12/2016 12:00:00 AM,81
1,1503960366,4/12/2016 1:00:00 AM,61
2,1503960366,4/12/2016 2:00:00 AM,59
3,1503960366,4/12/2016 3:00:00 AM,47
4,1503960366,4/12/2016 4:00:00 AM,48


In [62]:
dirtyHI = df_dict['hourlyIntensities']

In [63]:
dirtyHI.head()

,Id,ActivityHour,TotalIntensity,AverageIntensity
0,1503960366,4/12/2016 12:00:00 AM,20,0.333333
1,1503960366,4/12/2016 1:00:00 AM,8,0.133333
2,1503960366,4/12/2016 2:00:00 AM,7,0.116667
3,1503960366,4/12/2016 3:00:00 AM,0,0.000000
4,1503960366,4/12/2016 4:00:00 AM,0,0.000000


In [64]:
# I dont think the dataframes above will help our model, as the data is data we already have, just broken down into smaller
# time segments

In [65]:
dirtySD = df_dict['sleepDay']

In [66]:
dirtySD.head()

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712


In [67]:
dirtySD.shape

(413, 5)

In [68]:
for column_name in dirtySD.columns:
    column = dirtySD[column_name]
    # Get the count of Zeros in column 
    count = (column).isna().sum()
    print('Count of NAs in column ', column_name, ' is : ', count)

Count of NAs in column  Id  is :  0
Count of NAs in column  SleepDay  is :  0
Count of NAs in column  TotalSleepRecords  is :  0
Count of NAs in column  TotalMinutesAsleep  is :  0
Count of NAs in column  TotalTimeInBed  is :  0


In [69]:
# 24 users uploaded this data

dirtySD['Id'].nunique()

24

In [70]:
dirtySD.dtypes

Id                     int64
SleepDay              object
TotalSleepRecords      int64
TotalMinutesAsleep     int64
TotalTimeInBed         int64
dtype: object

In [71]:
dirtySD['Id'] = dirtySD.Id.astype(str)

In [72]:
# converting data types

dirtySD['SleepDay'] = pd.to_datetime(dirtySD['SleepDay'])

In [73]:
dirtySD.dtypes

Id                            object
SleepDay              datetime64[ns]
TotalSleepRecords              int64
TotalMinutesAsleep             int64
TotalTimeInBed                 int64
dtype: object

In [74]:
# merging sleeping data to our cleaned dataframe

cleanASHR = cleanAHR.merge(dirtySD, left_on= ['Id', 'ActivityDate'], right_on= ['Id', 'SleepDay'], how= 'left' )

In [75]:
cleanASHR.head()

,Id,ActivityDate,TotalSteps,TrackerDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,Time,Value,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,13162,8.50,1.88,0.55,6.06,0.0,25,13,328,728,1985,NaT,NaN,2016-04-12,1.0,327.0,346.0
1,1503960366,2016-04-13,10735,6.97,1.57,0.69,4.71,0.0,21,19,217,776,1797,NaT,NaN,2016-04-13,2.0,384.0,407.0
2,1503960366,2016-04-14,10460,6.74,2.44,0.40,3.91,0.0,30,11,181,1218,1776,NaT,NaN,NaT,NaN,NaN,NaN
3,1503960366,2016-04-15,9762,6.28,2.14,1.26,2.83,0.0,29,34,209,726,1745,NaT,NaN,2016-04-15,1.0,412.0,442.0
4,1503960366,2016-04-16,12669,8.16,2.71,0.41,5.04,0.0,36,10,221,773,1863,NaT,NaN,2016-04-16,2.0,340.0,367.0


In [76]:
# removing duplicated data rows

cleanASHR = cleanASHR.drop(['TotalSleepRecords', 'Time', 'SleepDay'], axis=1)

In [77]:
cleanASHR.head()

,Id,ActivityDate,TotalSteps,TrackerDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,Value,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12,13162,8.50,1.88,0.55,6.06,0.0,25,13,328,728,1985,NaN,327.0,346.0
1,1503960366,2016-04-13,10735,6.97,1.57,0.69,4.71,0.0,21,19,217,776,1797,NaN,384.0,407.0
2,1503960366,2016-04-14,10460,6.74,2.44,0.40,3.91,0.0,30,11,181,1218,1776,NaN,NaN,NaN
3,1503960366,2016-04-15,9762,6.28,2.14,1.26,2.83,0.0,29,34,209,726,1745,NaN,412.0,442.0
4,1503960366,2016-04-16,12669,8.16,2.71,0.41,5.04,0.0,36,10,221,773,1863,NaN,340.0,367.0


In [78]:
# creating dict to rename our columns

nmdict = {'Id': 'Id', 'ActivityDate': 'Date', 'TotalSteps': 'Steps', 'TrackerDistance': 'Distance', 'VeryActiveDistance': 'High_Activity_Dist', 'ModeratelyActiveDistance': 'Mod_Activity_dist', 'LightActiveDistance': 'Light_Activity_dist', 'SedentaryActiveDistance': 'Sed_Activity_Dist', 'VeryActiveMinutes': 'High_Activity_Min', 'FairlyActiveMinutes': 'Mod_Activity_Min', 'LightlyActiveMinutes': 'Light_Activity_Min', 'SedentaryMinutes': 'Sed_Min', 'Calories': 'Calories', 'Value': 'Heart_Rate', 'TotalMinutesAsleep': 'Time_Asleep', 'TotalTimeInBed': 'Time_In_Bed'}

In [79]:
cleanASHR = cleanASHR.rename(columns= nmdict)

In [80]:
cleanASHR.head()

,Id,Date,Steps,Distance,High_Activity_Dist,Mod_Activity_dist,Light_Activity_dist,Sed_Activity_Dist,High_Activity_Min,Mod_Activity_Min,Light_Activity_Min,Sed_Min,Calories,Heart_Rate,Time_Asleep,Time_In_Bed
0,1503960366,2016-04-12,13162,8.50,1.88,0.55,6.06,0.0,25,13,328,728,1985,NaN,327.0,346.0
1,1503960366,2016-04-13,10735,6.97,1.57,0.69,4.71,0.0,21,19,217,776,1797,NaN,384.0,407.0
2,1503960366,2016-04-14,10460,6.74,2.44,0.40,3.91,0.0,30,11,181,1218,1776,NaN,NaN,NaN
3,1503960366,2016-04-15,9762,6.28,2.14,1.26,2.83,0.0,29,34,209,726,1745,NaN,412.0,442.0
4,1503960366,2016-04-16,12669,8.16,2.71,0.41,5.04,0.0,36,10,221,773,1863,NaN,340.0,367.0


In [81]:
cleanASHR.describe()

,Steps,Distance,High_Activity_Dist,Mod_Activity_dist,Light_Activity_dist,Sed_Activity_Dist,High_Activity_Min,Mod_Activity_Min,Light_Activity_Min,Sed_Min,Calories,Heart_Rate,Time_Asleep,Time_In_Bed
count,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,318.000000,410.000000,410.000000
mean,8227.274971,5.912491,1.593737,0.612761,3.619365,0.001504,22.596945,14.700353,208.514689,954.754407,2358.641598,77.909016,419.268293,458.656098
std,4708.130555,3.699878,2.735995,0.907998,1.861216,0.005808,33.819395,20.547237,96.750526,281.872248,708.414490,10.394374,118.713834,127.534515
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.000000,59.377175,58.000000,61.000000
25%,4855.000000,3.345000,0.000000,0.000000,2.330000,0.000000,0.000000,0.000000,145.000000,720.500000,1853.500000,69.965015,361.000000,402.250000
50%,7924.000000,5.540000,0.370000,0.290000,3.550000,0.000000,6.000000,8.000000,206.000000,1020.000000,2211.000000,76.920131,432.500000,463.500000
75%,10880.000000,7.775000,2.145000,0.860000,4.880000,0.000000,33.000000,21.000000,270.000000,1190.500000,2830.000000,83.956209,491.500000,526.750000
max,36019.000000,28.030001,21.920000,6.480000,10.710000,0.070000,210.000000,143.000000,518.000000,1440.000000,4900.000000,109.789625,796.000000,961.000000


In [82]:
# removing the rows that showed a full 24 hours of sedentary activity

cleanASHR = cleanASHR[cleanASHR['Sed_Min'] != 1440]

In [83]:
# our cleaned data size

cleanASHR.shape

(844, 16)

In [84]:
# saving our cleaned data

cleanASHR.to_csv('cleanedFBdata.csv')